In [1]:
#Working ANN Model

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, SimpleRNN
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.optimizers import RMSprop
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.preprocessing.sequence import TimeseriesGenerator

# read in the data
deer = pd.read_csv("DMA1_withlatlong.csv")

In [3]:
deer = deer[['Status', 'Sample Date', 'Lat_Cent', 'Long_Cent','Age', 'Sex', 
             'Active - Hunter Killed', 'Active - Other', 'Active - Road-Killed',
             'Targeted - Other', 'Targeted-Clinical suspect']]
deer['Sample Date'] = pd.to_datetime(deer['Sample Date'])
deer['Age'] = pd.to_numeric(deer['Age'],errors='coerce')

In [4]:
deer = deer.dropna()

In [5]:
# features
X = deer.iloc[:,1:4]  
 
# with timestamp 
#X = deer.iloc[:,1:]  

# response
y = deer.Status


In [6]:
X["Sample Date"] = (X["Sample Date"]-X["Sample Date"].min()).astype('timedelta64[Y]').astype(int)+1


In [7]:
y=pd.DataFrame(y)
y.Status=y.Status.astype('float64')

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [9]:
from sklearn.utils import class_weight
y_train=y_train.values.reshape(7904,)

class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(y_train),
                                                 y_train)
class_weights = {i : class_weights[i] for i in range(2)}
class_weights

{0: 0.5036319612590799, 1: 69.33333333333333}

In [10]:
tf.random.set_seed(7)

In [11]:
# Simple neural net that DOES work (have to take out time stamp variable though, i.e. above)
# Uncomment to use

model = Sequential()
model.add(Flatten(input_shape=[3,]))
#model.add(keras.layers.BatchNormalization())
#model.add(Dense(150, activation='relu'))
#model.add(Dense(100, activation='relu'))
#model.add(Dense(30, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [12]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [13]:
sgd = keras.optimizers.SGD(lr=0.000001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy', f1_m, precision_m, recall_m])    

In [ ]:
history = model.fit(X_train, 
                    y_train,
                    epochs=20, 
                    batch_size=4,
                    class_weight=class_weights,
                    validation_data=(X_test, y_test), 
                    verbose=1)

Epoch 1/20
 390/1976 [====>.........................] - ETA: 1s - loss: 1.5342 - accuracy: 0.2612 - f1_m: 0.0071 - precision_m: 0.0044 - recall_m: 0.0176   